In [1]:
import torch
import torch.nn as nn
inputs = torch.Tensor(1,1,28,28)

In [2]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(777)
if device == "cuda":
    torch.cuda.manual_seed_all(777)

In [3]:
print(device)

cuda


In [4]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [5]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [6]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)


In [17]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        
        self.fc1 = nn.Linear(3*3*128,625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625,10,bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0),-1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [19]:
model = CNN().to(device)

value = (torch.Tensor(1,1,28,28)).to(device)
print(model(value).shape)

torch.Size([1, 10])


In [20]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [21]:
#training
total_batch = len(data_loader)
print("Learning Start...")
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X,Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost/total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning Start...
[Epoch:    1] cost = 0.166301236
[Epoch:    2] cost = 0.0440236069
[Epoch:    3] cost = 0.0307179093
[Epoch:    4] cost = 0.0237200446
[Epoch:    5] cost = 0.0197335687
[Epoch:    6] cost = 0.0135806249
[Epoch:    7] cost = 0.0128821451
[Epoch:    8] cost = 0.0117697828
[Epoch:    9] cost = 0.0102920169
[Epoch:   10] cost = 0.00722372765
[Epoch:   11] cost = 0.00791730545
[Epoch:   12] cost = 0.00752355251
[Epoch:   13] cost = 0.00645370362
[Epoch:   14] cost = 0.00389751
[Epoch:   15] cost = 0.00721739791
Learning Finished!


In [22]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy: ",accuracy.item())
    

Accuracy:  0.9784999489784241


D:\Anaconda\lib\site-packages\torchvision\datasets\mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
D:\Anaconda\lib\site-packages\torchvision\datasets\mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
